# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key
# geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,14.07,74,100,2.68,AU,1724698581
1,1,kodiak,57.7900,-152.4072,9.73,76,40,0.00,US,1724698279
2,2,tazacorte,28.6290,-17.9293,26.19,81,75,6.17,ES,1724698358
3,3,puerto natales,-51.7236,-72.4875,9.25,57,75,13.38,CL,1724698475
4,4,adamstown,-25.0660,-130.1015,18.27,76,100,8.74,PN,1724698269


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    size='Humidity',
    )

# Display the map
plt.savefig("output_data/Fig12.png")
map_plot_1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

<Figure size 640x480 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']<=25)\
                                    &(city_data_df['Max Temp']>=0)\
                                    &(city_data_df['Cloudiness']<=99)\
                                    &(city_data_df['Humidity']<80),:]


print(len(ideal_weather_df))

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

149


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,kodiak,57.7900,-152.4072,9.73,76,40,0.00,US,1724698279
3,3,puerto natales,-51.7236,-72.4875,9.25,57,75,13.38,CL,1724698475
5,5,ambovombe,-25.1667,46.0833,20.67,67,0,2.04,MG,1724698586
14,14,ayr,55.4627,-4.6339,15.99,77,40,4.63,GB,1724698597
15,15,changji,44.0167,87.3167,24.44,46,0,1.00,CN,1724698598


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,74,
1,kodiak,US,57.7900,-152.4072,76,
2,tazacorte,ES,28.6290,-17.9293,81,
3,puerto natales,CL,-51.7236,-72.4875,57,
4,adamstown,PN,-25.0660,-130.1015,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
hasaki - nearest hotel: Choshi Plaza Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
bilibino - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
sitka - nearest hotel: Aspen Suites Hotel
iqaluit - nearest hotel: Frobisher Inn
puerto natales - nearest hotel: Dorotea Patagonia Hostel
maun - nearest hotel: Center Lodge Conference Center
oranjemund - nearest hotel: Op My Stoep
kouango - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
albany - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
walvis bay - nearest hotel: Atlantic
east london - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
gadzhiyevo - nearest hotel: Оазис
inarajan village - nearest hotel: No hotel found
herma

<bound method NDFrame.head of                             City Country      Lat       Lng  Humidity  \
0                     bredasdorp      ZA -34.5322   20.0403        89   
1                         hasaki      JP  35.7333  140.8333        84   
2    edinburgh of the seven seas      SH -37.0676  -12.3116        63   
3                       belmonte      BR -15.8631  -38.8828        62   
4                   invercargill      NZ -46.4000  168.3500        86   
..                           ...     ...      ...       ...       ...   
541                    masterton      NZ -40.9597  175.6575        89   
542                       gannat      FR  46.1002    3.1989        59   
543                     kudymkar      RU  59.0173   54.6560        82   
544                 richards bay      ZA -28.7830   32.0377        39   
545             pinheiro machado      BR -31.5783  -53.3811        91   

                      Hotel Name  
0                 Victoria Hotel  
1             Choshi Pl

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
plt.savefig("output_data/Fig12.png")
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

<Figure size 640x480 with 0 Axes>